In [14]:
import requests
from bs4 import BeautifulSoup

# Scraper

Scraper to fetch article data online based on article information

In [ ]:
class Paper:
    def __init__(self, url):
        self.url = url
        self.authors = []
        self.title = ''

In [3]:
session = requests.session()

In [41]:
html = session.get('https://www.biorxiv.org/content/10.1101/2020.01.10.901801v1.full')

In [42]:
soup = BeautifulSoup(html.content, 'html.parser')

In [74]:
def biorxiv_html_paper_data_extractor(soup):
    '''
        Extract paper information from html webpage of biorxiv
    '''
    paper = {}
    paper['title'] = soup.h1.text
    
    # paper authors
    authors = []
    for author in soup.find_all('span',attrs={'class':'highwire-citation-author'}):
        author = author.find('span',attrs={'class':'nlm-given-names'}).text + ' ' + author.find('span',attrs={'class':'nlm-surname'}).text
        authors.append(author)
    #print(authors)
    paper['authors'] = authors
    
    sections = {}
    for section in soup.find_all('div',attrs={'class':'section'}):
        section_name = section.find('h2').text
        if section_name != 'References':
            #print(section_name)
            sections[section_name] = ''
            for p in section.find_all('p'):
                sections[section_name] += p.text + '\n'
            #print(sections[section_name])
            #print()
    paper['sections'] = sections
            
    # getting references data
    references = []
    for ref in soup.find('div',attrs={'class':'section ref-list'}).find_all('li'):
        reference = {}
        # label
        reference['label'] = ref.find('span',attrs={'class':'ref-label'}).text

        # authors
        author_names = []
        authors = ref.find('cite').find_all('span',attrs={'class':'cit-auth'})
        for author in authors:
            surname = author.find('span',attrs={'class':'cit-name-surname'}).text 
            given_names = author.find('span',attrs={'class':'cit-name-given-names'}).text
            author_name = surname + ' ' + given_names
            author_names.append(author_name)
        reference['authors'] = author_names

        # date
        reference['date'] = ref.find('cite').find('span',attrs={'class':'cit-pub-date'}).text

        # title
        reference['title'] = ref.find('cite').find('span',attrs={'class':'cit-article-title'}).text

        # links
        ref_links = []
        links = ref.find('div',attrs={'class':'cit-extra'}).find_all('a')
        for link in links:
            if link.text != 'OpenUrl':
                link_name = link.text
                link_url = link['href']
                ref_links.append((link_name,link_url))
        reference['links'] = ref_links
        references.append(reference)
    paper['references'] = references
    #print(references)
    return paper

In [75]:
paper_data = biorxiv_html_paper_data_extractor(soup)

In [76]:
paper_data

{'title': 'The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but essential for the production of infectious virus',
 'authors': ['Joseph C. Ward',
  'Lidia Lasecka-Dykes',
  'Chris Neil',
  'Oluwapelumi Adeyemi',
  'Sarah Gold',
  'Niall McLean',
  'Caroline Wright',
  'Morgan R. Herod',
  'David Kealy',
  'Emma Warner',
  'Donald P. King',
  'Tobias J. Tuthill',
  'David J. Rowlands',
  'Nicola J. Stonehouse',
  'Joseph C. Ward',
  'Lidia Lasecka-Dykes',
  'Chris Neil',
  'Oluwapelumi Adeyemi',
  'Sarah Gold',
  'Niall McLean',
  'Caroline Wright',
  'Morgan R. Herod',
  'David Kealy',
  'Emma Warner',
  'Donald P. King',
  'Tobias J. Tuthill',
  'David J. Rowlands',
  'Nicola J. Stonehouse',
  'Joseph C. Ward',
  'Lidia Lasecka-Dykes',
  'Chris Neil',
  'Oluwapelumi Adeyemi',
  'Sarah Gold',
  'Niall McLean',
  'Caroline Wright',
  'Morgan R. Herod',
  'David Kealy',
  'Emma Warner',
  'Donald P. King',
  'Tobias J. Tuthill',
  'David J. Rowla